In [4]:
from konlpy.tag import Okt
import pandas as pd
import random
import torch
from transformers import AutoTokenizer, AutoModel
from torch.utils.data import DataLoader
import numpy as np

d:\project\OSS_Project\AI\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# Okt 형태소 분석기 초기화
okt = Okt()

# 유의어 사전 정의
synonyms = {
    # 맛 관련
    '달달한': ['달콤한', '달달한', '달콤달콤한', '스위트한', '달큰한'],
    '부드러운': ['순한', '부드러운', '순하디순한', '소프트한', '매끄러운'],
    '강한': ['독한', '강력한', '세', '독하디독한', '알콜도수가 높은'],
    '상큼한': ['새콤한', '산뜻한', '신선한', '프레시한', '시원상큼한'],
    '쓴': ['씁쓸한', '쌉쌀한', '쓰디쓴', '씁슬한', '쌉싸름한'],
    
    # 행동 관련
    '마시': ['들이켜', '마셔', '음미하', '맛보', '즐기'],
    '찾': ['구하', '원하', '찾아보', '알아보', '검색하'],
    '추천': ['소개', '알려주', '추천', '제안', '안내'],
    
    # 문장 구조 변형용
    '~을 원해요': ['~이 마시고 싶어요', '~을 찾고 있어요', '~은 어떠세요?', '~을 추천해주세요'],
    '~이 있나요?': ['~을 찾고 있어요', '~을 구할 수 있을까요?', '~은 어디 있나요?']
}

In [6]:
def augment_korean_text(text, num_augments=2):
    augmented_texts = set()  # 중복 제거
    
    # 원본 문장의 형태소 분석
    morphs = okt.pos(text, norm=True, stem=True)
    
    while len(augmented_texts) < num_augments:
        new_text = []
        changed = False
        
        for word, pos in morphs:
            if pos in ['Adjective', 'Verb'] and word in synonyms:
                # 70% 확률로 유의어로 변경
                if random.random() < 0.7:
                    new_word = random.choice(synonyms[word])
                    new_text.append(new_word)
                    changed = True
                else:
                    new_text.append(word)
            else:
                new_text.append(word)
        
        # 문장 구조 변형 (30% 확률)
        if random.random() < 0.3:
            new_sentence = create_variant_structure(' '.join(new_text))
        else:
            new_sentence = ' '.join(new_text)
            
        # 원본과 다른 경우만 추가
        if changed and new_sentence != text:
            augmented_texts.add(new_sentence)
    
    return list(augmented_texts)

def create_variant_structure(text):
    templates = [
        "{}",
        "혹시 {}",
        "{} 있을까요?",
        "{}는 어떠세요?",
        "{} 마시고 싶네요."
    ]
    return random.choice(templates).format(text.rstrip('?.'))

# 데이터 로드 및 증강
df = pd.read_csv("data/raw_data.csv", encoding='utf-8')
augmented_data = {
    "입력 문장": [],
    "도수": [],
    "술 종류": [],
    "맛": []
}

# 데이터 증강 수행
for idx, row in df.iterrows():
    # 원본 데이터 추가
    augmented_data["입력 문장"].append(row["입력 문장"])
    augmented_data["도수"].append(row["도수"])
    augmented_data["술 종류"].append(row["술 종류"])
    augmented_data["맛"].append(row["맛"])
    
    # 증강 데이터 생성 및 추가
    augmented_texts = augment_korean_text(row["입력 문장"])
    for aug_text in augmented_texts:
        augmented_data["입력 문장"].append(aug_text)
        augmented_data["도수"].append(row["도수"])
        augmented_data["술 종류"].append(row["술 종류"])
        augmented_data["맛"].append(row["맛"])

# 결과 저장
augmented_df = pd.DataFrame(augmented_data)
augmented_df.to_csv("data/augmented_data.csv", index=False, encoding='utf-8')